In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

Idea: similarity is decided by how much percentage of their budget is used up.

| UserID | TipID1 | TipID2 | TipID3 | ... |
|--------|------|------|------|-----|
| thaer  | 0  | 0 | 1 | ... |
| erwoh  | 1 | 1 | 0 | ... |
| wobll  | 1 | 0 | 0 | ... |
| ...    | ...  | ...  | ...  | ... |


In [ ]:
tips = pd.read_csv("tips.csv")
Ratings = pd.read_csv("ratings.csv")
Tags = pd.read_csv("tags.csv")

Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

In [ ]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
check.head()

In [ ]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
final.head()

In [2]:
# Replacing NaN by Movie Average
final_tip = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

NameError: name 'final' is not defined

In [ ]:
# user similarity on replacing NAN by user avg
# b = cosine_similarity(final_user)
# np.fill_diagonal(b, 0)
# similarity_with_user = pd.DataFrame(b,index=final_user.index)
# similarity_with_user.columns=final_user.index
# similarity_with_user.head()

In [ ]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_tip)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

In [ ]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
# # top 30 neighbours for each user
# sim_user_30_u = find_n_neighbours(similarity_with_user,30)
# sim_user_30_u.head()

In [ ]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

In [ ]:
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

In [ ]:
a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
a.head()

In [ ]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [ ]:

score = User_item_score(320,7371)
print("score (u,i) is",score)

In [ ]:
Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))

In [ ]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

In [ ]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : 370")
print("   ")
for i in predicted_movies:
    print(i)

### References
- https://medium.com/sfu-big-data/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0
- https://github.com/ashaypathak/Recommendation-system/blob/master/Movie_Recommendation.ipynb
- https://en.wikipedia.org/wiki/Collaborative_filtering#Memory-based
- https://en.wikipedia.org/wiki/Locality-sensitive_hashing
- https://pypi.org/project/LocalitySensitiveHashing/
- https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/

### Ideas for future implementations

User-based collaborative filtering so that we will suggest tips based on how much of their budget has been spent (to hopefully avoid having similar tips being recommended to one user) <- also not sure that that is an issue yet. 
- Idea: similarity is decided by how much percentage of their budget is used up.

| UserID | Cat1 | Cat2 | Cat3 | ... |
|--------|------|------|------|-----|
| thaer  | .734 | .238 | 1.32 | ... |
| erwoh  | .781 | .362 | .987 | ... |
| wobll  | .294 | .272 | .549 | ... |
| ...    | ...  | ...  | ...  | ... |